In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
loan_df=pd.read_csv('Resources/lending_data.csv')

# Review the DataFrame
loan_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=loan_df['loan_status'].values.reshape(-1, 1)

# Separate the X variable, the features
X = loan_df.copy()
X.drop("loan_status", axis=1, inplace=True)

In [4]:
# Review the y variable Series
y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [5]:
# Review the X variable DataFrame
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
loan_df['loan_status'].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
#scaling the data to prepare for the model training

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_test_scaled[: 5]


array([[ 1.33383057,  1.32170668,  1.32215708,  1.53620797,  1.14084852,
         1.04334691,  1.32215708],
       [-0.00381349, -0.00491964, -0.00379305,  0.15702536,  0.09035056,
        -0.67289855, -0.00379305],
       [ 0.52168953,  0.53584922,  0.5337543 ,  0.7848731 ,  0.61559954,
         1.04334691,  0.5337543 ],
       [ 0.42614353,  0.41892623,  0.41429933,  0.65461753,  0.61559954,
         1.04334691,  0.41429933],
       [ 0.04395951,  0.06703297,  0.06787993,  0.24729423,  0.09035056,
        -0.67289855,  0.06787993]])

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)


# Fit the model using training data
classifier.fit(X_train_scaled, y_train)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.flatten()})



,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
19379,0,0
19380,0,0
19381,0,0
19382,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9889115309798473

In [12]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm_df



,Predicted 0,Predicted 1
Actual 0,18652,113
Actual 1,10,609


In [13]:
# Print the classification report for the model
from sklearn.metrics import classification_report
target_names = ["O", "1"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

           O       1.00      0.99      1.00     18765
           1       0.84      0.98      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression did very good job to predict the outcome of the loan using the given features. The model's overall accuracy is very high. It performed sligthly better in predicting healthy loan than high risk loan in terms of precision, recall and f1 score. This model is good enough to move to production phase based on the given metrics.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [14]:
!pip install imbalanced-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [32]:
# Import the RandomOverSampler module form imbalanced-learn
#from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE


# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
sm = SMOTE(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_SMOTE, y_train_SMOTE= sm.fit_resample(X_train_scaled, y_train)
X_train_SMOTE

array([[-0.57708952, -0.56367666, -0.5652314 , ..., -0.43489843,
        -0.67289855, -0.5652314 ],
       [-0.95927354, -0.98302549, -0.98332378, ..., -0.96014741,
        -0.67289855, -0.98332378],
       [ 0.09173251,  0.10413354,  0.10371642, ...,  0.09035056,
         1.04334691,  0.10371642],
       ...,
       [ 3.38806967,  3.37723713,  3.3767825 , ...,  3.24184446,
         2.75959236,  3.3767825 ],
       [ 4.10466471,  4.0985915 ,  4.0935123 , ...,  4.29234242,
         2.75959236,  4.0935123 ],
       [ 4.29575671,  4.28733876,  4.28464025, ...,  4.29234242,
         2.75959236,  4.28464025]])

In [34]:
# Count the distinct values of the resampled labels data

#number of samples for 0 (healthy loan) 
np.sum(y_train_SMOTE==0)

56271

In [31]:
#number of samples for 1 (high-risk loan) 
np.sum(y_train_SMOTE==1)


56271

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [35]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)


# Fit the model using the resampled training data
classifier.fit(X_train_SMOTE, y_train_SMOTE)


# Make a prediction using the testing data
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.flatten()})

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
19379,0,0
19380,0,0
19381,0,0
19382,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [37]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions)

0.9933850227303349

In [38]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm_df

,Predicted 0,Predicted 1
Actual 0,18638,127
Actual 1,4,615


In [39]:
# Print the classification report for the model
target_names = ["O", "1"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

           O       1.00      0.99      1.00     18765
           1       0.83      0.99      0.90       619

    accuracy                           0.99     19384
   macro avg       0.91      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 